In [1]:
!pip -q install PySastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.6/210.6 KB 4.6 MB/s eta 0:00:00


In [2]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 107.8 MB/s eta 0:00:00


In [5]:
import numpy as np
import pandas as pd
import random
import re
import string
import sys
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
import itertools
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from transformers import BertForSequenceClassification, BertConfig, BertTokenizer
from nltk.tokenize import TweetTokenizer
import torch.nn.functional as F
from tqdm import tqdm

OSError: [WinError 182] The operating system cannot run %1. Error loading "c:\Users\Ryzen\anaconda3\envs\fsb_streamlit\lib\site-packages\torch\lib\shm.dll" or one of its dependencies.

In [4]:
torch.cuda.empty_cache()

# Import df

In [5]:
!git clone https://github.com/mintup29/Hate-Speech-Recognition.git

Cloning into 'Hate-Speech-Recognition'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 5 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (5/5), 802.61 KiB | 2.73 MiB/s, done.


In [6]:
df = pd.read_csv('/content/Hate-Speech-Recognition/data.csv', encoding='latin-1')
alay_dict = pd.read_csv('/content/Hate-Speech-Recognition/new_kamusalay.csv', names = ['original', 'replacement'], encoding='latin-1')
stopword_dict = pd.read_csv('/content/Hate-Speech-Recognition/stopwordbahasa.csv', names = ['stopword'], encoding='latin-1')


In [7]:
df.head()

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong
0,- disaat semua cowok berusaha melacak perhatia...,1,1,1,0,0,0,0,0,1,1,0,0
1,RT USER: USER siapa yang telat ngasih tau elu?...,0,1,0,0,0,0,0,0,0,0,0,0
2,"41. Kadang aku berfikir, kenapa aku tetap perc...",0,0,0,0,0,0,0,0,0,0,0,0
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,0,0,0,0,0,0,0,0,0,0,0,0
4,USER USER Kaum cebong kapir udah keliatan dong...,1,1,0,1,1,0,0,0,0,0,1,0


In [8]:
alay_dict.head()

,original,replacement
0,anakjakartaasikasik,anak jakarta asyik asyik
1,pakcikdahtua,pak cik sudah tua
2,pakcikmudalagi,pak cik muda lagi
3,t3tapjokowi,tetap jokowi
4,3x,tiga kali


In [9]:
stopword_dict.head()

,stopword
0,ada
1,adalah
2,adanya
3,adapun
4,agak


# Data Preprocessing

In [10]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def lowercase(text):
    return text.lower()

def remove_unnecessary_char(text):
    text = re.sub('\n',' ',text) # Remove every '\n'
    text = re.sub('rt',' ',text) # Remove every retweet symbol
    text = re.sub('user',' ',text) # Remove every username
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))',' ',text) # Remove every URL
    text = re.sub('  +', ' ', text) # Remove extra spaces
    return text
    
def remove_nonaplhanumeric(text):
    text = re.sub('[^0-9a-zA-Z]+', ' ', text) 
    return text

alay_dict_map = dict(zip(alay_dict['original'], alay_dict['replacement']))
def normalize_alay(text):
    return ' '.join([alay_dict_map[word] if word in alay_dict_map else word for word in text.split(' ')])

def remove_stopword(text):
    text = ' '.join(['' if word in stopword_dict.stopword.values else word for word in text.split(' ')])
    text = re.sub('  +', ' ', text) # Remove extra spaces
    text = text.strip()
    return text

def stemming(text):
    return stemmer.stem(text)

In [11]:
def preprocess(text):
    text = lowercase(text) # 1
    text = remove_nonaplhanumeric(text) # 2
    text = remove_unnecessary_char(text) # 2
    text = normalize_alay(text) # 3
    text = stemming(text) # 4
    text = remove_stopword(text) # 5
    return text

def classify(hs, abs):
    hs_class = ""
    if int(hs) == 1 or int(abs) == 1:
        hs_class = 'positive'
    else:
        hs_class = 'negative'
    return hs_class

In [12]:
print("Toxic shape: ", df[(df['HS'] == 1) | (df['Abusive'] == 1)].shape)
print("Non-toxic shape: ", df[(df['HS'] == 0) & (df['Abusive'] == 0)].shape)

Toxic shape:  (7309, 13)
Non-toxic shape:  (5860, 13)


In [13]:
df['text'] = df['Tweet'].apply(preprocess)
df['hs_class'] = df.apply(lambda x: classify(x['HS'], x['Abusive']), axis=1)
df[['text', 'hs_class']].sample(10)

,text,hs_class
1491,nyata rezim anti islam bilang hoaks,positive
12190,orang malam bicara terang soe ahok gie,negative
5285,kualitas wakil rakyat pro rejim down grade duk...,negative
1874,tai,positive
12474,france england in the final duel imperialis ta...,negative
7932,tertawa mulu kunyuk,positive
12840,goreng goreng gosong suara murni manusia hubun...,positive
7774,negara demokrasi ya bubar komisi berantas korupsi,positive
8525,urus urus orang mulut nyinyir kayak banci lekong,positive
3071,ih najis belepotan,positive


In [14]:
df['hs_class'].value_counts()

positive    7309
negative    5860
Name: hs_class, dtype: int64

In [15]:
data = df[['hs_class', 'text']]
data.head()

,hs_class,text
0,positive,cowok usaha lacak perhati gue lantas remeh per...
1,positive,telat tau edan sarap gue gaul cigax jifla cal ...
2,negative,41 kadang pikir percaya tuhan jatuh kali kali ...
3,negative,ku tau mata sipit lihat
4,positive,kaum cebong kafir lihat dongok dungu haha


In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13169 entries, 0 to 13168
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   hs_class  13169 non-null  object
 1   text      13169 non-null  object
dtypes: object(2)
memory usage: 205.9+ KB


In [17]:
data_train_pos = data[data['hs_class']=='positive']
data_train_neg = data[data['hs_class']=='negative']

train_test_pos, validate_pos = train_test_split(data_train_pos, test_size=.2)
train_pos, test_pos = train_test_split(train_test_pos, test_size=.125)

train_test_neg, validate_neg = train_test_split(data_train_neg, test_size=.2)
train_neg, test_neg = train_test_split(train_test_neg, test_size=.125)

train_df = pd.concat([train_pos, train_neg])
validate_df = pd.concat([validate_pos, validate_neg])
test_df = pd.concat([test_pos, test_neg])

In [18]:
train_df.to_csv("train.csv", index=False)
validate_df.to_csv("validate.csv", index=False)
test_df.to_csv("test.csv", index=False)

In [19]:
train_df

,hs_class,text
12316,positive,nakal nakal ya besok pakai wkwk sarap
7606,positive,biadab kerja cina buang bendera indonesia tana...
9604,positive,calon calon marah bos langsung resign cupu
91,positive,transgender anjing uniform resource locator
12099,positive,media sampah berframing tim anies sandi kaget ...
...,...,...
5826,negative,teras saksi bisu ya maddie duh khawatir dicka nih
5034,negative,owner nya order sakit
8783,negative,pokok fisik pakai canda bentuk bulat sipit kul...
11425,negative,astagfirullah elok tuduh orang kafir


In [20]:
validate_df

,hs_class,text
52,positive,pinokio umur 3 2019 ganti presiden
1684,positive,marah umat gue pikir xf0 x9f x98 x80 xf0 x9f x...
621,positive,sabar gue adek adek kunyuk bau kayak sempak kuda
6781,positive,koruptor lindung komisi berantas korupsi bubar...
11178,positive,kesal gue komandan cai antek antek sok banget
...,...,...
1297,negative,sebar bahaya laten int
8275,negative,cita jaka a
6573,negative,golong asing untung tolong sepuh pasar gambar ...
7734,negative,sampah bu


In [21]:
test_df

,hs_class,text
7856,positive,siyaaaaap tolong kait kait imbal jasa jamin an...
3507,positive,blur kampret
10216,positive,oi ngantukkk banget bangkai suka masuk pagi xf...
9072,positive,ganyang partai komunis indonesia uniform resou...
4781,positive,cih najis
...,...,...
12889,negative,nama seba gede 2 000 orang beda seba haya hadi...
5881,negative,hutang ganti bahas halus gotong royong nalar w...
8062,negative,masuk jokowi tokoh muslim pengaruh kepala staf...
12976,negative,acara back to kampus fakultas ilmu budaya univ...


# Function Model

In [22]:
class DocumentSentimentDataset(Dataset):
    # Static constant variable
    LABEL2INDEX = {'positive': 1, 'negative': 0}
    INDEX2LABEL = {1: 'positive', 0: 'negative'}
    NUM_LABELS = 2
    
    def load_dataset(self, path): 
        df = pd.read_csv(path)
        # df.columns = ['sentiment','text']
        df['hs_class'] = df['hs_class'].apply(lambda lab: self.LABEL2INDEX[lab])
        return df
    
    def __init__(self, dataset_path, tokenizer, no_special_token=False, *args, **kwargs):
        self.data = self.load_dataset(dataset_path)
        self.tokenizer = tokenizer
        self.no_special_token = no_special_token
    
    def __getitem__(self, index):
        data = self.data.loc[index,:]
        text, hs_class = data['text'], data['hs_class']
        subwords = self.tokenizer.encode(text, add_special_tokens=not self.no_special_token)
        return np.array(subwords), np.array(hs_class), data['text']
    
    def __len__(self):
        return len(self.data)    
        
class DocumentSentimentDataLoader(DataLoader):
    def __init__(self, max_seq_len=512, *args, **kwargs):
        super(DocumentSentimentDataLoader, self).__init__(*args, **kwargs)
        self.collate_fn = self._collate_fn
        self.max_seq_len = max_seq_len
        
    def _collate_fn(self, batch):
        batch_size = len(batch)
        max_seq_len = max(map(lambda x: len(x[0]), batch))
        max_seq_len = min(self.max_seq_len, max_seq_len)
        
        subword_batch = np.zeros((batch_size, max_seq_len), dtype=np.int64)
        mask_batch = np.zeros((batch_size, max_seq_len), dtype=np.float32)
        sentiment_batch = np.zeros((batch_size, 1), dtype=np.int64)
        
        seq_list = []
        for i, (subwords, sentiment, raw_seq) in enumerate(batch):
            subwords = subwords[:max_seq_len]
            subword_batch[i,:len(subwords)] = subwords
            mask_batch[i,:len(subwords)] = 1
            sentiment_batch[i,0] = sentiment
            
            seq_list.append(raw_seq)
            
        return subword_batch, mask_batch, sentiment_batch, seq_list

In [23]:
def forward_sequence_classification(model, batch_data, i2w, is_test=False, device='cpu', **kwargs):
    # Unpack batch data
    if len(batch_data) == 3:
        (subword_batch, mask_batch, label_batch) = batch_data
        token_type_batch = None
    elif len(batch_data) == 4:
        (subword_batch, mask_batch, token_type_batch, label_batch) = batch_data
    
    # Prepare input & label
    subword_batch = torch.LongTensor(subword_batch)
    mask_batch = torch.FloatTensor(mask_batch)
    token_type_batch = torch.LongTensor(token_type_batch) if token_type_batch is not None else None
    label_batch = torch.LongTensor(label_batch)
    if device == "cuda":
        subword_batch = subword_batch.cuda()
        mask_batch = mask_batch.cuda()
        token_type_batch = token_type_batch.cuda() if token_type_batch is not None else None
        label_batch = label_batch.cuda()

    # Forward model
    outputs = model(subword_batch, attention_mask=mask_batch, token_type_ids=token_type_batch, labels=label_batch)
    loss, logits = outputs[:2]
    # generate prediction & label list
    list_hyp = []
    list_label = []
    hyp = torch.topk(logits, 1)[1]
    for j in range(len(hyp)):
        list_hyp.append(i2w[hyp[j].item()])
        list_label.append(i2w[label_batch[j][0].item()])
        
    return loss, list_hyp, list_label

In [24]:
def document_sentiment_metrics_fn(list_hyp, list_label):
    metrics = {}
    metrics["ACC"] = accuracy_score(list_label, list_hyp)
    metrics["F1"] = f1_score(list_label, list_hyp, average='macro')
    metrics["REC"] = recall_score(list_label, list_hyp, average='macro')
    metrics["PRE"] = precision_score(list_label, list_hyp, average='macro')
    return metrics

In [25]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']
    
def count_param(module, trainable=False):
    if trainable:
        return sum(p.numel() for p in module.parameters() if p.requires_grad)
    else:
        return sum(p.numel() for p in module.parameters())

def metrics_to_string(metric_dict):
    string_list = []
    for key, value in metric_dict.items():
        string_list.append('{}:{:.2f}'.format(key, value))
    return ' '.join(string_list)

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
set_seed(26092020)

# Pretrained Model

In [26]:
# Load Tokenizer and Config
tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-base-p1')
config = BertConfig.from_pretrained('indobenchmark/indobert-base-p1')
config.num_labels = DocumentSentimentDataset.NUM_LABELS

# Instantiate model
model = BertForSequenceClassification.from_pretrained('indobenchmark/indobert-base-p1', config=config)

# tokenizer = BertTokenizer.from_pretrained("indobenchmark/indobert-base-p1")
# model = BertForMultiLabelClassification.from_pretrained('indobenchmark/indobert-base-p1', config=config)

'''
if you are loading saved model
loaded_model = torch.load("./model.pkl")
model = loaded_model
'''
# model

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


'\nif you are loading saved model\nloaded_model = torch.load("./model.pkl")\nmodel = loaded_model\n'

In [70]:
train_dataset_path = "/content/train.csv"
test_dataset_path = "/content/test.csv"
valid_dataset_path = "/content/validate.csv"

train_na = pd.read_csv(train_dataset_path).dropna().to_csv("train_new.csv", index=False)
test_na = pd.read_csv(test_dataset_path).dropna().to_csv("test_new.csv", index=False)
valid_na = pd.read_csv(valid_dataset_path).dropna().to_csv("validate_new.csv", index=False)

train_dataset_path = './train_new.csv'
test_dataset_path = './test_new.csv'
valid_dataset_path = './validate_new.csv'

train_dataset = pd.read_csv(train_dataset_path)
test_dataset = pd.read_csv(test_dataset_path)
valid_dataset = pd.read_csv(valid_dataset_path)

In [66]:
# train_dataset_path = './train.csv'
# test_dataset_path = './test.csv'
# valid_dataset_path = './validate.csv'

# train_dataset = pd.read_csv(train_dataset_path).dropna().reset_index(drop=True)
# test_dataset = pd.read_csv(test_dataset_path).dropna().reset_index(drop=True)
# valid_dataset = pd.read_csv(valid_dataset_path).dropna().reset_index(drop=True)

In [71]:
train_dataset = DocumentSentimentDataset(train_dataset_path, tokenizer, lowercase=True)
test_dataset = DocumentSentimentDataset(test_dataset_path, tokenizer, lowercase=True)
valid_dataset = DocumentSentimentDataset(valid_dataset_path, tokenizer, lowercase=True)

train_loader = DocumentSentimentDataLoader(dataset=train_dataset, max_seq_len=512, batch_size=16, num_workers=16, shuffle=True)  
test_loader = DocumentSentimentDataLoader(dataset=test_dataset, max_seq_len=512, batch_size=16, num_workers=16, shuffle=False)
valid_loader = DocumentSentimentDataLoader(dataset=valid_dataset, max_seq_len=512, batch_size=16, num_workers=16, shuffle=False)  


w2i, i2w = DocumentSentimentDataset.LABEL2INDEX, DocumentSentimentDataset.INDEX2LABEL
print(w2i)
print(i2w)

{'positive': 1, 'negative': 0}
{1: 'positive', 0: 'negative'}


In [58]:
text = 'Woi dasar kau anjing babi antek komunis'
subwords = tokenizer.encode(text)
subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)

logits = model(subwords)[0]
label = torch.topk(logits, k=1, dim=-1)[1].squeeze().item()

print(f'Text: {text} | Label : {i2w[label]} ({F.softmax(logits, dim=-1).squeeze()[label] * 100:.3f}%)')

Text: Woi dasar kau anjing babi antek komunis | Label : positive (58.368%)


In [59]:
text = 'Budi pergi ke pondok indah mall membeli cakwe'
subwords = tokenizer.encode(text)
subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)

logits = model(subwords)[0]
label = torch.topk(logits, k=1, dim=-1)[1].squeeze().item()

print(f'Text: {text} | Label : {i2w[label]} ({F.softmax(logits, dim=-1).squeeze()[label] * 100:.3f}%)')

Text: Budi pergi ke pondok indah mall membeli cakwe | Label : negative (51.899%)


# Tuning Model

In [32]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [68]:
from torch import optim
optimizer = optim.Adam(model.parameters(), lr=3e-6)
model = model.cuda()

In [72]:
# Train
n_epochs = 10
for epoch in range(n_epochs):
    model.train()
    torch.set_grad_enabled(True)
 
    total_train_loss = 0
    list_hyp, list_label = [], []

    train_pbar = tqdm(train_loader, leave=True, total=len(train_loader))
    for i, batch_data in enumerate(train_pbar):
        # Forward model
        loss, batch_hyp, batch_label = forward_sequence_classification(model, batch_data[:-1], i2w=i2w, device='cuda')

        # Update model
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        tr_loss = loss.item()
        total_train_loss = total_train_loss + tr_loss

        # Calculate metrics
        list_hyp += batch_hyp
        list_label += batch_label

        train_pbar.set_description("(Epoch {}) TRAIN LOSS:{:.4f} LR:{:.8f}".format((epoch+1),
            total_train_loss/(i+1), get_lr(optimizer)))

    # Calculate train metric
    metrics = document_sentiment_metrics_fn(list_hyp, list_label)
    print("(Epoch {}) TRAIN LOSS:{:.4f} {} LR:{:.8f}".format((epoch+1),
        total_train_loss/(i+1), metrics_to_string(metrics), get_lr(optimizer)))

    # Evaluate on validation
    model.eval()
    torch.set_grad_enabled(False)
    
    total_loss, total_correct, total_labels = 0, 0, 0
    list_hyp, list_label = [], []

    pbar = tqdm(valid_loader, leave=True, total=len(valid_loader))
    for i, batch_data in enumerate(pbar):
        batch_seq = batch_data[-1]        
        loss, batch_hyp, batch_label = forward_sequence_classification(model, batch_data[:-1], i2w=i2w, device='cuda')
        
        # Calculate total loss
        valid_loss = loss.item()
        total_loss = total_loss + valid_loss

        # Calculate evaluation metrics
        list_hyp += batch_hyp
        list_label += batch_label
        metrics = document_sentiment_metrics_fn(list_hyp, list_label)

        pbar.set_description("VALID LOSS:{:.4f} {}".format(total_loss/(i+1), metrics_to_string(metrics)))
        
    metrics = document_sentiment_metrics_fn(list_hyp, list_label)
    print("(Epoch {}) VALID LOSS:{:.4f} {}".format((epoch+1),
        total_loss/(i+1), metrics_to_string(metrics)))

(Epoch 1) TRAIN LOSS:0.4178 LR:0.00000300: 100%|██████████| 574/574 [01:35<00:00,  6.04it/s]


(Epoch 1) TRAIN LOSS:0.4178 ACC:0.81 F1:0.80 REC:0.80 PRE:0.81 LR:0.00000300


VALID LOSS:0.3203 ACC:0.87 F1:0.87 REC:0.87 PRE:0.87: 100%|██████████| 165/165 [00:12<00:00, 12.74it/s]


(Epoch 1) VALID LOSS:0.3203 ACC:0.87 F1:0.87 REC:0.87 PRE:0.87


(Epoch 2) TRAIN LOSS:0.2727 LR:0.00000300: 100%|██████████| 574/574 [01:34<00:00,  6.06it/s]


...

(Epoch 9) VALID LOSS:0.4433 ACC:0.89 F1:0.89 REC:0.89 PRE:0.89


(Epoch 10) TRAIN LOSS:0.0314 LR:0.00000300: 100%|██████████| 574/574 [01:35<00:00,  5.99it/s]


(Epoch 10) TRAIN LOSS:0.0314 ACC:0.99 F1:0.99 REC:0.99 PRE:0.99 LR:0.00000300


VALID LOSS:0.4741 ACC:0.89 F1:0.89 REC:0.89 PRE:0.89: 100%|██████████| 165/165 [00:12<00:00, 13.27it/s]

(Epoch 10) VALID LOSS:0.4741 ACC:0.89 F1:0.89 REC:0.89 PRE:0.89


In [73]:
correct, incorrect = 0, 0
for i in range(10):
    print('='*40)
#     print()
    single_row = data.sample()
    test_text = single_row['text'].values[0]
    test_res = single_row['hs_class'].values[0]
    print(f'test sentence:\n{test_text}\n')
    print(f'actual class : {test_res}')

    text = test_text

    subwords = tokenizer.encode(text)
    subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)

    logits = model(subwords)[0]
    labels = [torch.topk(logit, k=1, dim=-1)[1].squeeze().item() for logit in logits]
    
    for i, label in enumerate(labels):
        print(f'Text: {text} | Label : {i2w[label]} ({F.softmax(logits, dim=-1).squeeze()[label] * 100:.3f}%)')
        if(i2w[label]==test_res):
            print("> CORRECT PREDICTION!\n")
            correct+=1
        else:
            print("> INCORRECT PREDICTION!\n")
            incorrect+=1
print(f'\n\ncorrect result: {correct}/10')
print(f'incorect result: {incorrect}/10')

test sentence:
haha sis main sarah nih agenda yahudi xf0 x9f x98 x82

actual class : negative
Text: haha sis main sarah nih agenda yahudi xf0 x9f x98 x82 | Label : negative (99.976%)
> CORRECT PREDICTION!

test sentence:
pilih kepala daerah curang jawa barat jawa barat kantong komunis catat ajar marhaen lahir daerah kubur nya marhaen daerah

actual class : negative
Text: pilih kepala daerah curang jawa barat jawa barat kantong komunis catat ajar marhaen lahir daerah kubur nya marhaen daerah | Label : negative (99.906%)
> CORRECT PREDICTION!

test sentence:
pasu nasi bungkus grak xf0 x9f x98 x8

actual class : negative
Text: pasu nasi bungkus grak xf0 x9f x98 x8 | Label : negative (99.963%)
> CORRECT PREDICTION!

test sentence:
benah alon alon mu do mayak kick off blayer blayer wira wiri bawa genderuwo nama geng xe2 x80 xa6

actual class : negative
Text: benah alon alon mu do mayak kick off blayer blayer wira wiri bawa genderuwo nama geng xe2 x80 xa6 | Label : negative (99.963%)
> CORRE

In [74]:
def return_prediction_result(text):
    subwords = tokenizer.encode(text)
    subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)

    logits = model(subwords)[0]
    labels = [torch.topk(logit, k=1, dim=-1)[1].squeeze().item() for logit in logits]

    for i, label in enumerate(labels):
        return i2w[label]

In [75]:
test_data = pd.read_csv(test_dataset_path)
total_len, correct = len(test_data.values), 0
for row in test_data.values:
    actual = row[0]
    text = row[1]
    pred = (return_prediction_result(text))
    if actual == pred:
        correct+=1
print(f'Score based on test data: {correct}/{total_len}  Percentage:{round(correct/total_len * 100,2)}%')

Score based on test data: 1164/1313  Percentage:88.65%


# Saving Model

In [76]:
torch.save(model, "model.pkl")
loaded_model = torch.load("./model.pkl")
single_row = data.sample()
test_text = single_row['text'].values[0]
test_res = single_row['hs_class'].values[0]
print(f'test sentence:\n{test_text}\n')
print(f'actual class : {test_res}')

text = test_text

subwords = tokenizer.encode(text)
subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)

logits = loaded_model(subwords)[0]
labels = [torch.topk(logit, k=1, dim=-1)[1].squeeze().item() for logit in logits]

for i, label in enumerate(labels):
    print(f'Text: {text} | Label : {i2w[label]} ({F.softmax(logits, dim=-1).squeeze()[label] * 100:.3f}%)')

test sentence:
hai guys sebentar agustus lupa bundar ya tanggal tanggal 12 agustus 2018 yuk seru seru bareng kocak aula nada kak mila armidya sari kiwil lutfi afifih cungkring arjuna jawa si kribo jimat

actual class : negative
Text: hai guys sebentar agustus lupa bundar ya tanggal tanggal 12 agustus 2018 yuk seru seru bareng kocak aula nada kak mila armidya sari kiwil lutfi afifih cungkring arjuna jawa si kribo jimat | Label : negative (99.977%)
